In [1]:
!pip install pydicom 
!pip install pylibjpeg[all]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np

from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.optimizers import Adam

import config
from model import resnet
from data_generator.data_generator import DataGenerator
from data_generator.helper import excel

In [9]:
pos, neg = excel.get_names(config.PATIENT_LIST_PATH)

In [10]:
neg = neg[:len(pos)]

In [11]:
train_names_pos, val_names_pos = tts(pos, test_size=0.2, random_state=42)
train_names_neg, val_names_neg = tts(neg, test_size=0.2, random_state=42)

In [12]:
train_names = np.asarray(train_names_pos + train_names_neg)
np.random.shuffle(train_names)
val_names = np.asarray(val_names_pos + val_names_neg)
np.random.shuffle(val_names)

In [19]:
train_gen = DataGenerator(train_names, "train", config.PATIENT_LIST_PATH, batch_size=4)
val_gen = DataGenerator(val_names, "train", config.PATIENT_LIST_PATH, batch_size=4)

In [3]:
model = resnet.build_resnet_18(input_shape=(100, 100, 100, 1), num_outputs=2, reg_factor=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])

In [21]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=config.LOG_PATH)

In [ ]:
model.fit(train_gen, batch_size=4, epochs=50, validation_data=val_gen, callbacks = [tensorboard], verbose=2)

Epoch 1/50
12/12 - 652s - loss: 1.0031 - accuracy: 0.9583 - val_loss: 51.7287 - val_accuracy: 0.5000 - 652s/epoch - 54s/step
Epoch 2/50
12/12 - 572s - loss: 0.9475 - accuracy: 1.0000 - val_loss: 18.6511 - val_accuracy: 0.5000 - 572s/epoch - 48s/step
Epoch 3/50
12/12 - 629s - loss: 0.9333 - accuracy: 1.0000 - val_loss: 5.8881 - val_accuracy: 0.5000 - 629s/epoch - 52s/step
Epoch 4/50
12/12 - 567s - loss: 0.9173 - accuracy: 1.0000 - val_loss: 1.7421 - val_accuracy: 0.6667 - 567s/epoch - 47s/step
Epoch 5/50
12/12 - 572s - loss: 0.9005 - accuracy: 1.0000 - val_loss: 0.9413 - val_accuracy: 1.0000 - 572s/epoch - 48s/step
Epoch 6/50
12/12 - 626s - loss: 0.8833 - accuracy: 1.0000 - val_loss: 0.8795 - val_accuracy: 1.0000 - 626s/epoch - 52s/step
Epoch 7/50
12/12 - 617s - loss: 0.8660 - accuracy: 1.0000 - val_loss: 0.8582 - val_accuracy: 1.0000 - 617s/epoch - 51s/step
Epoch 8/50
12/12 - 625s - loss: 0.8487 - accuracy: 1.0000 - val_loss: 0.8401 - val_accuracy: 1.0000 - 625s/epoch - 52s/step
Epoch 

In [ ]:
model.save(config.MODEL_PATH)